In [ ]:
import numpy as np
import math
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters import QTradableStocksUS

from quantopian.pipeline.factors import Latest
from quantopian.pipeline.data import morningstar
from quantopian.pipeline.factors import CustomFactor, SimpleMovingAverage, AverageDollarVolume, Returns, RSI
from quantopian.pipeline.classifiers.morningstar import Sector
from quantopian.pipeline.data.zacks import EarningsSurprises

import talib
import pandas as pd
import pyfolio as pf
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import linear_model, decomposition, ensemble, preprocessing, isotonic, metrics

bs = morningstar.balance_sheet
cfs = morningstar.cash_flow_statement
is_ = morningstar.income_statement
or_ = morningstar.operation_ratios
er = morningstar.earnings_report
v = morningstar.valuation
vr = morningstar.valuation_ratios

In [ ]:
def make_pipeline():
    testing_factor = vr.pe_ratio.latest
    universe = (QTradableStocksUS() & testing_factor.notnull())
    testing_factor = testing_factor.rank(mask=universe, method='average')
    
    pipe = Pipeline(columns={
        'testing_factor': testing_factor
    },screen= universe)
    
    return pipe

result = run_pipeline(make_pipeline(), start_date='2015-01-01',end_date='2016-01-01')
result.head()
    

In [ ]:
assets = result.index.levels[1].unique()
len(assets)

In [ ]:
pricing_data = get_pricing(assets,start_date='2014-12-01',end_date='2016-02-01', fields='open_price')

In [ ]:
import alphalens as al
from alphalens.utils import get_clean_factor_and_forward_returns
from alphalens.tears import create_returns_tear_sheet

factor_data = get_clean_factor_and_forward_returns(
    result['testing_factor'], # How to analyze a specific pipeline column with Alphalens
    pricing_data,
    quantiles = 2,
    periods=(1,5,10)
)
create_returns_tear_sheet(factor_data=factor_data)